In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150

In [112]:
import numpy as np
import utils
import tempfile

from utils import mackey_glass

import tensorflow as tf
from temporal_basis_transformation_network.keras import TemporalBasisTrafo
from temporal_basis_transformation_network import Inverse

import dlop_ldn_function_bases as bases

In [36]:
# Create a long MG dataset
dt, T, tau = 0.1, 10000.0, 17.0
xs = mackey_glass(int(T / dt), tau, dt=dt, rng=np.random.RandomState(1904))

In [37]:
# Chop up the data into a bunch of training samples
N_wnd = int(np.ceil(tau / dt) + 1)
N_pred = 512 # Number of samples to predict
N_smpls = 100000 # Number of samples to generate
N_batch = 100
smpls_x = np.zeros((N_smpls, N_wnd))
smpls_t = np.zeros((N_smpls, N_pred))
for i in range(N_smpls):
    i0 = np.random.randint(N_smpls - N_wnd - N_pred)
    i1, i2 = i0 + N_wnd, i0 + N_wnd + N_pred
    smpls_x[i] = xs[i0:i1]
    smpls_t[i] = xs[i1:i2]

N_test = 20000
i_train0, i_train1 = 0, N_smpls - 2 * N_test
i_val0, i_val1 = N_smpls - 2 * N_test, N_smpls - N_test
i_test0, i_test1 = N_smpls - N_test, N_smpls

ds_train = tf.data.Dataset.from_tensor_slices((
            smpls_x[i_train0:i_train1], smpls_t[i_train0:i_train1]))
ds_train = ds_train.shuffle(i_train1 - i_train0)
ds_train = ds_train.batch(N_batch)

ds_val = tf.data.Dataset.from_tensor_slices((
            smpls_x[i_val0:i_val1], smpls_t[i_val0:i_val1]))
ds_val = ds_val.batch(N_batch)

ds_test = tf.data.Dataset.from_tensor_slices((
            smpls_x[i_test0:i_test1], smpls_t[i_test0:i_test1]))
ds_test = ds_test.batch(N_batch)

In [109]:
N_units = 1
N_neurons = 500
q = 16
H = bases.mk_haar_basis(q, N_wnd)
model = tf.keras.models.Sequential([
  tf.keras.layers.Reshape((N_wnd, 1)),                   # (N_wnd, 1)
  TemporalBasisTrafo(H, n_units=N_units, pad=False),     # (1, q)
  tf.keras.layers.Dense(N_neurons, activation='relu'),   # (1, N_neurons)
  tf.keras.layers.Dropout(0.5),                          # (1, N_neurons)
  tf.keras.layers.Dense(N_neurons, activation='relu'),   # (1, N_neurons)
  tf.keras.layers.Dropout(0.5),                          # (1, N_neurons)
  tf.keras.layers.Dense(N_pred, use_bias=False),         # (1, N_wnd)
  tf.keras.layers.Reshape((N_pred,))                     # (N_pred)
])


In [110]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.mse,
    metrics=['mse',]
)

In [111]:
model.fit(
    ds_train,
    epochs=20,
    validation_data=ds_val,
)

Epoch 1/20
600/600 [==============================] - 6s 9ms/step - loss: 0.1362 - mse: 0.1362 - val_loss: 0.0085 - val_mse: 0.0085
Epoch 2/20
600/600 [==============================] - 6s 9ms/step - loss: 0.0177 - mse: 0.0177 - val_loss: 0.0040 - val_mse: 0.0040
Epoch 3/20
600/600 [==============================] - 6s 9ms/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.0029 - val_mse: 0.0029
Epoch 4/20
600/600 [==============================] - 6s 9ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.0020 - val_mse: 0.0020
Epoch 5/20
258/600 [===========>..................] - ETA: 2s - loss: 0.0094 - mse: 0.0094

KeyboardInterrupt: 

In [ ]:
rows, cols = 5, 5
fig, axs = plt.subplots(rows, cols, sharex=True, sharey=True, figsize=(5.0, 5.0))
for i, ax in enumerate(axs.flat):
    ax.plot(np.arange(0, N_wnd), smpls_x[i])
    ax.plot(np.arange(N_wnd, N_wnd + N_pred), smpls_t[i])
    ax.plot(np.arange(N_wnd, N_wnd + N_pred), model(smpls_x[i].reshape(1, -1)).numpy()[0])
    for spine in ['left', 'bottom', 'top', 'right']:
        ax.spines[spine].set_visible(False)
    ax.set_yticks([])
    ax.set_xticks([])

In [122]:
ds_train.element_spec[1].shape[1]

512